In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

import json

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

from torchvision.models import resnet50, resnet18, resnext50_32x4d

from datetime import datetime as dt

import imgaug as ia
import imgaug.augmenters as iaa

from dataset_functions import *
from trainer import *

import tkinter as tk
from PIL import Image, ImageTk
import torchvision.transforms as T

# file containing the dataset (.npy file)
DATASET_FILE = 'training_all_colour_256.npy'

# set to True if running for the first time & there are no trained weights to load
DOWNLOAD_WEIGHTS = False
# choose what type of model to use from the pytorch models
model = resnet18(pretrained=DOWNLOAD_WEIGHTS)

WEIGHT_FILE = 'weights/resnet18_weights_alldata.pth'

# pick a model (resnet50, resnet18)
MODEL_NAME = 'resnext50_32x4d'

LOG_NAME = MODEL_NAME + '_alldata_scheduled_10e128bsMSE_Adam'
WEIGHT_DIR = 'weights'
LOG_DIR = 'logs'


c:\Users\gameg\miniconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\gameg\miniconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
c:\Users\gameg\miniconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [2]:
# dynamically define whether to run on gpu or cpu
device_to_use = torch.device("cpu")

if torch.cuda.is_available():
    device_to_use = torch.device("cuda:0")
    print("Running on GPU")
else:
    print("Running on CPU")

Running on GPU


Prepare the Dataset:

In [ ]:
im_size = 256

# imgaug transforms
seq = iaa.Sequential([
    iaa.Resize({"height": im_size, "width": im_size}),
])

train_X, train_y, test_X, test_y, im_info = load_split_data(DATASET_FILE, seq, validation_percent=0.1)

num_classes = im_info['classes']
im_chan = im_info['channels']
im_size = im_info['size']

print(len(train_X), len(train_y))

Working with the model:

In [ ]:
# load from resnet modelinternet
print('Downloading ImageNet weights:', DOWNLOAD_WEIGHTS)

# model setup

# Get the number of inputs to the last fully connected layer in the model
num_ftrs = model.fc.in_features
# create a new final fully connected layer that we can train to replace the other fully connected layer
model.fc = nn.Linear(num_ftrs, num_classes)

# load weights from file
weight_file = WEIGHT_FILE

if not DOWNLOAD_WEIGHTS: # load from file
    model.load_state_dict(torch.load(weight_file, map_location=device_to_use))

print('Done loading weights')

# add the model to the device
model = model.to(device_to_use)

cow_model = Trainer(model, im_size, LOG_NAME, device=device_to_use)

# if there is a memory error, lower the batch size (you want a batch size of at least 8 however)
cow_model.BATCH_SIZE = 128
cow_model.EPOCHS = 10
cow_model.learning_rate = 0.001

cow_model.log_dir = LOG_DIR

cow_model.loss_function = nn.MSELoss()
cow_model.optimizer = optim.Adam(model.fc.parameters(), lr=cow_model.learning_rate)
# See: https://coderzcolumn.com/tutorials/artificial-intelligence/pytorch-learning-rate-schedules#2
# step_size is after how many epochs/batches to update the LR
# gamma is how much to multiply the learning rate by at each step
cow_model.scheduler = lr_scheduler.StepLR(cow_model.optimizer, step_size=2, gamma=0.9, verbose=True)

print(f"Created {MODEL_NAME}, batch size={cow_model.BATCH_SIZE}, learning rate={cow_model.learning_rate}, training for {cow_model.EPOCHS} epochs using {cow_model.optimizer} optimizer & {cow_model.loss_function} loss...")

In [ ]:
plt.imshow(train_X[0].permute(1,2,0))
plt.show()

In [ ]:
# Transfer learning

# freeze all the layers
for param in model.parameters():
    param.requires_grad = False

# unfreeze the final fully connected layer
model.fc.requires_grad_(True)

val_acc, val_loss, res, lab = cow_model.test(test_X=test_X, test_y=test_y)

print(f'Before Training: acc={val_acc}, loss={val_loss}')

cow_model.train(train_X, train_y, validate=True, val_steps=50, test_X=test_X, test_y=test_y)


# print('Fine-tuning')
# # Fine-tune for one epoch:

# # unfreeze all the layers
# for param in model.parameters():
#     param.requires_grad = True

# # train for 1 epoch
# cow_model.EPOCHS = 1
# cow_model.modelname=LOG_NAME+'_2',
# cow_model.train(train_X, train_y, validate=True, val_steps=50, test_X=test_X, test_y=test_y)


In [ ]:
val_acc, val_loss, bad_results, bad_labels = cow_model.test(test_X=test_X, test_y=test_y, size=(int(len(test_y)-1)))

print(f'After Training: acc={val_acc}, loss={val_loss}')
print(f'{len(bad_results)} incorrect predictions')


In [ ]:
len(train_X)

In [ ]:
# Save the model: https://towardsdatascience.com/everything-you-need-to-know-about-saving-weights-in-pytorch-572651f3f8de

model_out_path = fix_filename(f'{LOG_NAME}_weights_{dt.datetime.now().isoformat()}.pth')

# make the directory in case
if not os.path.isdir(WEIGHT_DIR):
    os.mkdir(WEIGHT_DIR)

# save the model weights (the state_dict)
# NOTE: Must be loaded to an already-defined resnet50 architecture
torch.save(model.state_dict(), os.path.join(WEIGHT_DIR, model_out_path))

In [ ]:
# UI that shows all incorrect predictions & the PREDICTED label from the test set

trans = T.ToPILImage()

bad_images = [trans(i.cpu()) for i in bad_results]
bad_labels = [i.cpu() for i in bad_labels]

LABELS = {"0": "EMPTY", "1": "COW"}

class app():

    im_to_show = 0

    def __init__(self, x, y):
        self.root = tk.Tk()
        self.canvas = tk.Canvas(self.root, width=x, height=y)
        self.canvas.pack()

        Bu = tk.Button(self.root, text ="^", command = self.upim)
        Bd = tk.Button(self.root, text ="v", command = self.downim)

        Bu.pack()
        Bd.pack()

        # self.canvas.create_image((x/2)-60,y/2,anchor=tk.NW,image=self.img)
        # self.canvas.create_text(x/2, (y/2)-10, text=self.txt, fill="black", font=('Helvetica 12 bold'))
        self.canvas.pack

        self.root.mainloop()

    def upim(self):
        self.im_to_show = (self.im_to_show + 1) % len(bad_images)
        txt = LABELS[str(bad_labels[self.im_to_show].item())]

        self.canvas.delete('all')
        img = ImageTk.PhotoImage(bad_images[self.im_to_show])

        self.canvas.create_image((x/2)-60,y/2,anchor=tk.NW,image=img)
        self.canvas.create_text(x/2, (y/2)-10, text=txt, fill="black", font=('Helvetica 12 bold'))
        # for some unknown reason, the below line NEEDS to be here, even though it doesnt work
        self.canvas.create_text(x/2, (y/2)-10, text=self.txt, fill="black", font=('Helvetica 12 bold'))


    def downim(self):
        self.im_to_show = self.im_to_show - 1 if self.im_to_show - 1 >= 0 else len(bad_images) - 1
        txt = LABELS[str(bad_labels[self.im_to_show].item())]

        self.canvas.delete('all')
        img = ImageTk.PhotoImage(bad_images[self.im_to_show])

        self.canvas.create_image((x/2)-60,y/2,anchor=tk.NW,image=img)
        self.canvas.create_text(x/2, (y/2)-10, text=txt, fill="black", font=('Helvetica 12 bold'))
        # for some unknown reason, the below line NEEDS to be here, even though it doesnt work
        self.canvas.create_text(x/2, (y/2)-10, text=self.txt, fill="black", font=('Helvetica 12 bold'))

x, y = 400, 400

app(x, y)